In [ ]:
#| default_exp population

## 02_Population

> holds multiple individuals and their meta data

In [ ]:
#| hide
import attr
import torch
from burbankai.individual import *


In [ ]:
@attr.s(auto_attribs=True)
class Population:
    """Manages a collection of individuals."""
    individuals: list[Individual] = attr.Factory(list)
    sim_param: 'SimParam' = None

    def __len__(self):
        return len(self.individuals)
    
    def generate_random_founders(self, n_founders: int):
        """Generates a founder population with random haplotypes."""
        assert n_founders > 0, "Number of founders must be greater than zero."
        for i in range(n_founders):
            haplotypes = torch.randint(
                0, 2, (self.sim_param.ploidy, self.sim_param.n_chromosomes, self.sim_param.n_loci_per_chr)
            )
            individual = Individual(haplotypes=haplotypes, id=i + 1)
            self.individuals.append(individual)

    def set_phenotypes(self, varE: torch.Tensor):
            """Sets phenotypes for all individuals in the population."""
            n_traits = self.sim_param.n_traits
            assert varE.size(0) == n_traits, f"varE should have the same number of elements as traits ({n_traits})"
            for individual in self.individuals:
                assert individual.phenotype is None, "Phenotypes have already been set for this individual."
                assert individual.genetic_values is None, "Genetic values have already been set for this individual."
                genetic_value = self.calculate_genetic_value(individual)
                environmental_noise = torch.randn(n_traits) * torch.sqrt(varE)
                individual.phenotype = genetic_value + environmental_noise
                individual.genetic_values = genetic_value

    def calculate_genetic_value(self, individual: Individual):
            """Calculates the genetic value for an individual for all traits."""
            assert self.sim_param is not None, "Simulation parameters must be set before calculating genetic values."
            total_genetic_value = torch.zeros(self.sim_param.n_traits)
            for trait_idx, trait in enumerate(self.sim_param.traits):
                assert trait.qtl_effects.size(1) == self.sim_param.n_chromosomes, "Trait QTL effects should have the same number of chromosomes as the simulation parameters."
                assert trait.qtl_effects.size(2) <= self.sim_param.n_loci_per_chr, "Trait QTL effects should have the same or fewer number of loci per chromosome as the simulation parameters."
                genetic_value = trait.mean
                for chr in range(self.sim_param.n_chromosomes):
                    for ploidy_idx in range(self.sim_param.ploidy):
                        for qtl_idx in range(trait.n_qtl_per_chr):
                            qtl_location = trait.qtl_locations[ploidy_idx, chr, qtl_idx]
                            allele_dosage = individual.allele_dosage[chr, qtl_location]
                            effect = trait.qtl_effects[ploidy_idx, chr, qtl_idx]
                            genetic_value += allele_dosage * effect
                total_genetic_value[trait_idx] = genetic_value # Assign the genetic value for each trait to the correct index
            return total_genetic_value
